### Продвинутый SQL. Часть 2. Аналитика.
- Входные данные: базой данных StackOverflow — сервиса вопросов и ответов о программировании.

In [2]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://pictures.s3.yandex.net/resources/Frame_353_1_1665399433.png")

1.
Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT date_trunc('month', creation_date)::date as month,
       sum(views_count) as views
FROM stackoverflow.posts
GROUP BY month
ORDER BY views DESC

2.
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
WITH s1 AS (SELECT  p.creation_date AS date_post,
        t.type,
        u.creation_date AS date_user,
        p.user_id,
        display_name
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id
JOIN stackoverflow.post_types t ON t.id = p.post_type_id
WHERE (DATE_TRUNC('day', p.creation_date) <= (DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month')) AND type = 'Answer'), 

s2 AS (SELECT COUNT(DISTINCT(user_id)) AS sum_id,
       display_name,
       COUNT(type) AS sum_answers
       FROM s1
       GROUP BY display_name)

SELECT sum_id, display_name
FROM s2
WHERE sum_answers > 100
;

3.
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH s1 AS (SELECT DATE_TRUNC('MONTH', p.creation_date)::date AS post_pub,
       p.id AS post_id,
       u.id AS user_id,
       DATE_TRUNC('MONTH', u.creation_date)::date as month
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE u.id IN (SELECT u.id
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
WHERE DATE_TRUNC('MONTH', u.creation_date) = '2008-09-01'
AND DATE_TRUNC('MONTH', p.creation_date) = '2008-12-01'))
SELECT post_pub, COUNT(post_id)
FROM s1
GROUP BY post_pub
ORDER BY post_pub DESC
;

4.
Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts 
ORDER BY user_id,
         creation_date
;

5.
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
SELECT ROUND(CAST(COUNT(dt) AS NUMERIC) / count(distinct (user_id)))
FROM (SELECT DISTINCT(dt), user_id
FROM (SELECT DATE_TRUNC('DAY', creation_date)::DATE dt, user_id
FROM stackoverflow.posts
WHERE (DATE_TRUNC('DAY', creation_date)::DATE BETWEEN '2008-12-01' AND '2008-12-07')) AS s1
GROUP BY user_id, dt) AS s2;

6.
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? 

Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
WITH s1 AS (SELECT DISTINCT EXTRACT(MONTH FROM creation_date) as cd, count(id) AS posts
FROM stackoverflow.posts
WHERE EXTRACT(MONTH FROM creation_date) BETWEEN '09' AND '12'
GROUP BY cd)
SELECT *, 
      ROUND(CAST((posts - LAG(posts, 1, NULL) OVER (ORDER BY cd)) AS numeric) / (LAG(posts, 1, NULL) OVER (ORDER BY cd)) *100, 2) AS percent
FROM s1
;

7.
Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT DISTINCT(week), LAST_VALUE(creation_date) OVER (PARTITION BY week ORDER BY week) as last_p
FROM (SELECT EXTRACT(WEEK FROM creation_date) as week, creation_date
FROM stackoverflow.posts
WHERE DATE_TRUNC('MONTH',creation_date) = '2008-10-01' AND
user_id IN (
    SELECT user_id
    FROM stackoverflow.posts
    GROUP BY user_id
    ORDER BY count(id) DESC
    LIMIT 1)) AS s2
GROUP BY week,creation_date;